In [5]:
import pygame
import heapq
import random
import time



In [6]:
# Define colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
RED = (255, 0, 0)
BLUE = (0, 0, 255)
BROWN = (165, 42, 42)
GRAY = (128, 128, 128)
YELLOW = (255, 255, 0)

# Define grid size
GRID_SIZE = 15

# Define window size
WINDOW_SIZE = [600, 600]

# Define cell size
CELL_SIZE = WINDOW_SIZE[0] // GRID_SIZE

# Define obstacle probability
OBSTACLE_PROBABILITY = 0.3  # Increased probability



In [7]:
# Define delivery locations
DELIVERY_LOCATIONS = []
for _ in range(5):
    x = random.randint(0, GRID_SIZE - 1)
    y = random.randint(0, GRID_SIZE - 1)
    DELIVERY_LOCATIONS.append((x, y))

# Initialize pygame
pygame.init()

# Set up the window
window = pygame.display.set_mode(WINDOW_SIZE)
pygame.display.set_caption("Autonomous Delivery Robot")

# Define the grid
grid = [[0 for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]



In [8]:
# Generate obstacles and cars
for i in range(GRID_SIZE):
    for j in range(GRID_SIZE):
        if random.random() < OBSTACLE_PROBABILITY:
            grid[i][j] = random.choice([2, 3, 4])  # Representing different types of obstacles
        elif random.random() < 0.05:  # Adjust this probability as needed
            grid[i][j] = 5  # Representing cars



In [9]:
# Ensure robot and delivery locations are not blocked
grid[0][0] = 0  # Robot initial location
for loc in DELIVERY_LOCATIONS:
    grid[loc[0]][loc[1]] = 0



In [10]:
# Define the heuristic function
def heuristic(a, b):
    (x1, y1) = a
    (x2, y2) = b
    return abs(x1 - x2) + abs(y1 - y2)

# Define the A* algorithm
def astar(start, goal):
    frontier = []
    heapq.heappush(frontier, (0, start))
    came_from = {}
    cost_so_far = {}
    came_from[start] = None
    cost_so_far[start] = 0

    while frontier:
        current = heapq.heappop(frontier)[1]

        if current == goal:
            path = []
            while current != start:
                path.append(current)
                current = came_from[current]
            path.append(start)
            path.reverse()
            return path

        for next in [(current[0] + 1, current[1]), (current[0] - 1, current[1]),
                     (current[0], current[1] + 1), (current[0], current[1] - 1)]:
            if 0 <= next[0] < GRID_SIZE and 0 <= next[1] < GRID_SIZE and grid[next[0]][next[1]] == 0:
                new_cost = cost_so_far[current] + 1
                if next not in cost_so_far or new_cost < cost_so_far[next]:
                    cost_so_far[next] = new_cost
                    priority = new_cost + heuristic(next, goal)
                    heapq.heappush(frontier, (priority, next))
                    came_from[next] = current

    return []



In [11]:
def main():
    global DELIVERY_LOCATIONS
    running = True
    path = []
    start = (0, 0)

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # Clear the window
        window.fill(WHITE)

        # Draw the grid
        for i in range(GRID_SIZE):
            for j in range(GRID_SIZE):
                rect = pygame.Rect(j * CELL_SIZE, i * CELL_SIZE, CELL_SIZE, CELL_SIZE)
                if grid[i][j] == 1:  # Normal grid
                    pygame.draw.rect(window, WHITE, rect, 1)
                elif grid[i][j] == 2:  # Trees
                    pygame.draw.rect(window, GREEN, rect)
                elif grid[i][j] == 3:  # Buildings
                    pygame.draw.rect(window, GRAY, rect)
                elif grid[i][j] == 4:  # Houses
                    pygame.draw.rect(window, BROWN, rect)
                elif grid[i][j] == 5:  # Cars
                    pygame.draw.rect(window, YELLOW, rect)

        # Draw the delivery locations
        for loc in DELIVERY_LOCATIONS:
            pygame.draw.circle(window, BLUE, (loc[1] * CELL_SIZE + CELL_SIZE // 2, loc[0] * CELL_SIZE + CELL_SIZE // 2), CELL_SIZE // 4)

        # Draw the robot
        pygame.draw.circle(window, RED, (start[1] * CELL_SIZE + CELL_SIZE // 2, start[0] * CELL_SIZE + CELL_SIZE // 2), CELL_SIZE // 3)

        # Draw the path
        if path:
            for i in range(len(path) - 1):
                pygame.draw.line(window, BLUE, (path[i][1] * CELL_SIZE + CELL_SIZE // 2, path[i][0] * CELL_SIZE + CELL_SIZE // 2),
                                 (path[i + 1][1] * CELL_SIZE + CELL_SIZE // 2, path[i + 1][0] * CELL_SIZE + CELL_SIZE // 2), 3)

        # Update the display
        pygame.display.flip()

        # Deliver items
        if DELIVERY_LOCATIONS:
            current_goal = min(DELIVERY_LOCATIONS, key=lambda loc: heuristic(start, loc))
            path = astar(start, current_goal)
            if path:
                while path:
                    next_step = path.pop(0)
                    pygame.draw.line(window, BLUE, (start[1] * CELL_SIZE + CELL_SIZE // 2, start[0] * CELL_SIZE + CELL_SIZE // 2),
                                     (next_step[1] * CELL_SIZE + CELL_SIZE // 2, next_step[0] * CELL_SIZE + CELL_SIZE // 2), 3)
                    start = next_step
                    pygame.draw.circle(window, RED, (start[1] * CELL_SIZE + CELL_SIZE // 2, start[0] * CELL_SIZE + CELL_SIZE // 2), CELL_SIZE // 3)
                    pygame.display.flip()
                    time.sleep(0.3)
                DELIVERY_LOCATIONS.remove(current_goal)
        else:
            # Return to initial location
            path_back = astar(start, (0, 0))
            if path_back:
                while path_back:
                    next_step = path_back.pop(0)
                    pygame.draw.line(window, BLUE, (start[1] * CELL_SIZE + CELL_SIZE // 2, start[0] * CELL_SIZE + CELL_SIZE // 2),
                                     (next_step[1] * CELL_SIZE + CELL_SIZE // 2, next_step[0] * CELL_SIZE + CELL_SIZE // 2), 3)
                    start = next_step
                    pygame.draw.circle(window, RED, (start[1] * CELL_SIZE + CELL_SIZE // 2, start[0] * CELL_SIZE + CELL_SIZE // 2), CELL_SIZE // 3)
                    pygame.display.flip()
                    time.sleep(0.3)
                    if not path_back:
                        DELIVERY_LOCATIONS.clear()  # All deliveries completed, clear the delivery locations list
            running = False  # Stop the loop once all deliveries are completed

    pygame.quit()



In [12]:
if __name__ == "__main__":
    main() 